In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pylab
import scipy.stats as stat
from scipy.stats import ttest_ind, chi2_contingency
import statsmodels.api as sm
from statsmodels.formula.api import ols
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import os

In [43]:
os.chdir("C:\\Users\\Riya\\Documents\\Codesoft\\Credit Card Fraud Detection\\Credit-Card-Spam-Detection")
df = pd.read_csv("cardfraud.csv")
df.head(5)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


### Identifying the dimensions

In [44]:
df.shape

(555719, 23)

In [45]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

### Variable Identification

In [46]:
df.dtypes

Unnamed: 0                 int64
trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object

In [47]:
categorical = []
continuous = []
for column in df.columns:
    if df[column].dtypes == "int64" or df[column].dtypes == "float64":
        continuous.append(column)
    if df[column].dtypes == "object":
        categorical.append(column)

In [48]:
categorical

['trans_date_trans_time',
 'merchant',
 'category',
 'first',
 'last',
 'gender',
 'street',
 'city',
 'state',
 'job',
 'dob',
 'trans_num']

In [49]:
continuous

['Unnamed: 0',
 'cc_num',
 'amt',
 'zip',
 'lat',
 'long',
 'city_pop',
 'unix_time',
 'merch_lat',
 'merch_long',
 'is_fraud']

### Univarient Analysis for Contineous Variables

In [50]:
df.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05,5.557190e+05,555719.000000,555719.000000,555719.000000
mean,277859.000000,4.178387e+17,69.392810,48842.628015,38.543253,-90.231325,8.822189e+04,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,1.309837e+18,156.745941,26855.283328,5.061336,13.721780,3.003909e+05,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,1.800429e+14,9.630000,26292.000000,34.668900,-96.798000,7.410000e+02,1.376029e+09,34.755302,-96.905129,0.000000
50%,277859.000000,3.521417e+15,47.290000,48174.000000,39.371600,-87.476900,2.408000e+03,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,4.635331e+15,83.010000,72011.000000,41.894800,-80.175200,1.968500e+04,1.385867e+09,41.954163,-80.264637,0.000000
max,555718.000000,4.992346e+18,22768.110000,99921.000000,65.689900,-67.950300,2.906700e+06,1.388534e+09,66.679297,-66.952026,1.000000


### Univarient Analysis for Categorical Variables

In [51]:
categorical

['trans_date_trans_time',
 'merchant',
 'category',
 'first',
 'last',
 'gender',
 'street',
 'city',
 'state',
 'job',
 'dob',
 'trans_num']

In [52]:
for column in categorical:
    print("--------------------------------")
    print(column)
    print("--------------------------------")
    print(df[column].value_counts()/len(df[column]))
    print("--------------------------------")
    print("\n")

--------------------------------
trans_date_trans_time
--------------------------------
trans_date_trans_time
2020-12-19 16:02:22    0.000007
2020-12-13 17:53:47    0.000007
2020-12-17 20:36:39    0.000007
2020-10-05 19:37:49    0.000007
2020-12-27 01:22:37    0.000005
                         ...   
2020-08-25 13:15:57    0.000002
2020-08-25 13:15:37    0.000002
2020-08-25 13:15:22    0.000002
2020-08-25 13:15:12    0.000002
2020-12-31 23:59:34    0.000002
Name: count, Length: 544760, dtype: float64
--------------------------------


--------------------------------
merchant
--------------------------------
merchant
fraud_Kilback LLC                        0.003345
fraud_Cormier LLC                        0.002874
fraud_Schumm PLC                         0.002809
fraud_Kuhn LLC                           0.002737
fraud_Dickinson Ltd                      0.002733
                                           ...   
fraud_Treutel-King                       0.000581
fraud_Satterfield-Lowe   

In [53]:
df.drop(['trans_date_trans_time'], axis=1, inplace=True)

### Bivarient Analyisis contineous - contineous variable

In [54]:
df.dtypes

Unnamed: 0      int64
cc_num          int64
merchant       object
category       object
amt           float64
first          object
last           object
gender         object
street         object
city           object
state          object
zip             int64
lat           float64
long          float64
city_pop        int64
job            object
dob            object
trans_num      object
unix_time       int64
merch_lat     float64
merch_long    float64
is_fraud        int64
dtype: object

In [55]:
df

,Unnamed: 0,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,555714,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,555715,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,555716,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,555717,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [56]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(['cc_num'], axis=1, inplace=True)
df.drop(['zip'], axis=1, inplace=True)
df.drop(['lat'], axis=1, inplace=True)
df.drop(['long'], axis=1, inplace=True)
df.drop(['city_pop'], axis=1, inplace=True)
df.drop(['merch_lat'], axis=1, inplace=True)
df.drop(['merch_long'], axis=1, inplace=True)

In [57]:
df

,merchant,category,amt,first,last,gender,street,city,state,job,dob,trans_num,unix_time,is_fraud
0,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,SC,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,0
1,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,UT,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,0
2,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,NY,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,0
3,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,FL,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,0
4,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,MI,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,MO,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,0
555715,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,TX,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,0
555716,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,WA,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,0
555717,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,ID,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,0


In [58]:
df.columns

Index(['merchant', 'category', 'amt', 'first', 'last', 'gender', 'street',
       'city', 'state', 'job', 'dob', 'trans_num', 'unix_time', 'is_fraud'],
      dtype='object')

### Bivarient Analyisis categorical - categorical variable

In [59]:
df.dtypes

merchant      object
category      object
amt          float64
first         object
last          object
gender        object
street        object
city          object
state         object
job           object
dob           object
trans_num     object
unix_time      int64
is_fraud       int64
dtype: object

In [61]:
chi2_contingency(pd.crosstab(df['merchant'],df['is_fraud']))[1]

1.8022270460261968e-205

In [62]:
chi2_contingency(pd.crosstab(df['gender'],df['is_fraud']))[1]

0.5922844718406959

In [63]:
chi2_contingency(pd.crosstab(df['street'],df['is_fraud']))[1]

0.0

In [64]:
df.drop(['street'], axis=1, inplace=True)

In [65]:
chi2_contingency(pd.crosstab(df['city'],df['is_fraud']))[1]

0.0

In [66]:
df.drop(['city'], axis=1, inplace=True)

In [67]:
chi2_contingency(pd.crosstab(df['job'],df['is_fraud']))[1]

0.0

In [68]:
df.drop(['job'], axis=1, inplace=True)

In [69]:
chi2_contingency(pd.crosstab(df['dob'],df['is_fraud']))[1]

0.0

In [70]:
df.drop(['dob'], axis=1, inplace=True)

In [71]:
chi2_contingency(pd.crosstab(df['trans_num'],df['is_fraud']))[1]

0.4993693095753912

In [72]:
df

,merchant,category,amt,first,last,gender,state,trans_num,unix_time,is_fraud
0,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,SC,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,0
1,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,UT,324cc204407e99f51b0d6ca0055005e7,1371816873,0
2,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,NY,c81755dbbbea9d5c77f094348a7579be,1371816893,0
3,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,FL,2159175b9efe66dc301f149d3d5abf8c,1371816915,0
4,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,MI,57ff021bd3f328f8738bb535c302a31b,1371816917,0
...,...,...,...,...,...,...,...,...,...,...
555714,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,MO,9b1f753c79894c9f4b71f04581835ada,1388534347,0
555715,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,TX,2090647dac2c89a1d86c514c427f5b91,1388534349,0
555716,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,WA,6c5b7c8add471975aa0fec023b2e8408,1388534355,0
555717,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,ID,14392d723bb7737606b2700ac791b7aa,1388534364,0


### Missing Value treatement

In [73]:
df.describe()

,amt,unix_time,is_fraud
count,555719.000000,5.557190e+05,555719.000000
mean,69.392810,1.380679e+09,0.003860
std,156.745941,5.201104e+06,0.062008
min,1.000000,1.371817e+09,0.000000
25%,9.630000,1.376029e+09,0.000000
50%,47.290000,1.380762e+09,0.000000
75%,83.010000,1.385867e+09,0.000000
max,22768.110000,1.388534e+09,1.000000


In [74]:
df.isnull().sum()

merchant     0
category     0
amt          0
first        0
last         0
gender       0
state        0
trans_num    0
unix_time    0
is_fraud     0
dtype: int64

In [75]:
df['merchant'].value_counts()

merchant
fraud_Kilback LLC                        1859
fraud_Cormier LLC                        1597
fraud_Schumm PLC                         1561
fraud_Kuhn LLC                           1521
fraud_Dickinson Ltd                      1519
                                         ... 
fraud_Treutel-King                        323
fraud_Satterfield-Lowe                    319
fraud_Kessler Group                       318
fraud_Jerde-Hermann                       312
fraud_Ritchie, Bradtke and Stiedemann     304
Name: count, Length: 693, dtype: int64

In [76]:
df['category'].value_counts()

category
gas_transport     56370
grocery_pos       52553
home              52345
shopping_pos      49791
kids_pets         48692
shopping_net      41779
entertainment     40104
personal_care     39327
food_dining       39268
health_fitness    36674
misc_pos          34574
misc_net          27367
grocery_net       19426
travel            17449
Name: count, dtype: int64

In [77]:
df['state'].value_counts()

state
TX    40393
NY    35918
PA    34326
CA    24135
OH    20147
MI    19671
IL    18960
FL    18104
AL    17532
MO    16501
MN    13719
AR    13484
NC    12868
SC    12541
KY    12506
VA    12506
WI    12370
IN    11959
IA    11819
OK    11379
GA    11277
MD    11152
WV    10838
NJ    10528
NE    10257
KS     9943
LA     8988
MS     8833
WY     8454
WA     8116
OR     7811
TN     7359
NM     7020
ME     6928
ND     6397
CO     5886
SD     5250
MA     5186
MT     5052
VT     5044
UT     4658
AZ     4592
NH     3449
CT     3277
ID     2490
NV     2451
DC     1517
HI     1090
AK      843
RI      195
Name: count, dtype: int64

In [78]:
df['first'].value_counts()

first
Christopher    11443
Robert          9076
Jessica         8655
David           8599
Michael         8530
               ...  
Mike             199
Dennis            12
Evan              12
Bruce             10
Nicole             9
Name: count, Length: 341, dtype: int64

In [79]:
df['last'].value_counts()

last
Smith        12146
Williams     10056
Davis         9524
Johnson       8556
Rodriguez     7485
             ...  
Santiago        12
Bates           12
Robbins         10
Freeman          8
Bartlett         7
Name: count, Length: 471, dtype: int64

### Exporting Feature Selected Data

In [80]:
df.to_csv('CSD_FS.csv', encoding='utf-8', index=False)